# Lab 1 Part II: UCI-HAR 1D CNN

## Imports

In [2]:
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, Activation
from keras.utils import get_file
from keras.utils import to_categorical

2024-03-08 14:38:31.108777: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 14:38:31.176235: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 14:38:31.176325: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 14:38:31.176386: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 14:38:31.188592: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 14:38:31.189661: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

## Load and format UCI-HAR dataset (raw data)

In [3]:
dataset_path = get_file(None, "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip", extract=True, file_hash="53e099237392e0b9602f8c38f578bd8f") # Download, cache and extract UCI-HAR
dataset_dir = Path(dataset_path).parent

def load_set(dataset_dir, part: str): # Load separate sensor signals and combine them into a single array, load labels separately
    data = np.hstack([np.loadtxt(dataset_dir/'UCI HAR Dataset'/part/'Inertial Signals'/f'{sensor}_{axis}_{part}.txt')
                for sensor in ('body_acc', 'body_gyro', 'total_acc')
                    for axis in ('x', 'y', 'z')]).reshape((-1, 128, 9))
    labels = to_categorical(np.loadtxt(dataset_dir/'UCI HAR Dataset'/part/f'y_{part}.txt') - 1)
    return data, labels

x_train, y_train = load_set(dataset_dir, 'train')
x_test, y_test = load_set(dataset_dir, 'test')

   8192/Unknown - 0s 0us/step

## Export complete test dataset (2947 vectors)

In [4]:
np.savetxt('x_test_uci-har.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_uci-har.csv', y_test, delimiter=',', fmt='%s')

## Export small test dataset (250 vectors)

In [5]:
x_test_250 = x_test[0:250]
y_test_250 = y_test[0:250]
np.savetxt('x_test_uci-har_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_uci-har_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model

In [6]:
model = Sequential()
model.add(Input(shape=(128, 9)))
model.add(Conv1D(filters=2, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(units=6))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 126, 2)            56        
                                                                 
 flatten (Flatten)           (None, 252)               0         
                                                                 
 dense (Dense)               (None, 6)                 1518      
                                                                 
 activation (Activation)     (None, 6)                 0         
                                                                 
Total params: 1574 (6.15 KB)
Trainable params: 1574 (6.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train model

In [7]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

Epoch 1/3


2024-03-08 14:38:50.685223: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33878016 exceeds 10% of free system memory.


 48/230 [=====>........................] - ETA: 0s - loss: 0.8744 - categorical_accuracy: 0.6191  

2024-03-08 14:38:51.406449: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28862568 exceeds 10% of free system memory.
2024-03-08 14:38:51.411126: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28862568 exceeds 10% of free system memory.
2024-03-08 14:38:51.414509: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28862568 exceeds 10% of free system memory.
2024-03-08 14:38:51.417835: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28862568 exceeds 10% of free system memory.


230/230 [==============================] - 2s 5ms/step - loss: 0.5447 - categorical_accuracy: 0.7714 - val_loss: 0.5154 - val_categorical_accuracy: 0.7896
Epoch 2/3
230/230 [==============================] - 1s 4ms/step - loss: 0.3259 - categorical_accuracy: 0.8719 - val_loss: 0.4006 - val_categorical_accuracy: 0.8368
Epoch 3/3
230/230 [==============================] - 1s 4ms/step - loss: 0.2900 - categorical_accuracy: 0.8848 - val_loss: 0.3381 - val_categorical_accuracy: 0.8575


## Evaluate model on complete test dataset

In [8]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

93/93 - 0s - loss: 0.3381 - categorical_accuracy: 0.8575 - 182ms/epoch - 2ms/step
93/93 [==============================] - 0s 2ms/step
tf.Tensor(
[[484   1  11   0   0   0]
 [ 90 341  40   0   0   0]
 [ 42  70 308   0   0   0]
 [  0   2   0 386 103   0]
 [  1   0   0  60 471   0]
 [  0   0   0   0   0 537]], shape=(6, 6), dtype=int32)


## Evaluate model on small test dataset

In [9]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 0.2922 - categorical_accuracy: 0.8400 - 35ms/epoch - 4ms/step
8/8 [==============================] - 0s 2ms/step
tf.Tensor(
[[53  0  0  0  0  0]
 [ 0 25  0  0  0  0]
 [ 3 15  6  0  0  0]
 [ 0  0  0 24 22  0]
 [ 0  0  0  0 54  0]
 [ 0  0  0  0  0 48]], shape=(6, 6), dtype=int32)


## Save trained model

In [10]:
model.save('lab1_part2_uci-har.h5')

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
